In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
data_path = '../data/ignore/'
lap_data = 'Laptop-partially-cleaned.csv'
pd1 = pd.read_csv(data_path + lap_data, dtype=str)

In [4]:
# Process laptop data

pd_out = pd1.copy()

In [5]:
# The CPU and GPU will be processed later with cpu.csv and gpu.csv

# Display: including size, resolution, refresh rate, panel type, refresh sync
display = pd1['Display'].str.split(',', expand=True).to_numpy()
for row in display:
    if row[3] is None:
        if row[2] is not None and not((row[2].find('Hz') != -1 or row[2].isnumeric())):
            row[3] = row[2]
            row[2] = None
display = pd.DataFrame(display)
# dealing with column 0, minor error in some collumns, convert them all to float
display0 = display[0].str.extract(r'(\d+\.?\d+)', expand=True).astype(float)
display0.rename(columns={0: 'Display Size'}, inplace=True)
display0.fillna(15.6, inplace=True)
# dealing with column 1, since resolution quality approximates its length*height, convert them all to float
display1 = display[1].str.extract(r'(\d+)\s?x\s?(\d+)', expand=True).astype(float)
display1.rename(columns={0: 'Resolution: width', 1: 'Resolution: height'}, inplace=True)
display1['Resolution: height'].fillna(1080, inplace=True)
display1['Resolution: width'].fillna(1920, inplace=True)
# dealing with column 2, extract refresh rate and convert them all to float
display2 = display[2].str.extract(r'(\d+)', expand=True).astype(float)
display2.rename(columns={0: 'Refresh Rate'}, inplace=True)
# fill NaN with 60, since 90 is the min in the known data, I assmue 60 is the missing since it's usually ignore in information sites
display2.fillna(60, inplace=True)
# dealing with column 3, leave alone
display3 = display[3]
display3.fillna('IPS', inplace=True)
display3.rename('Panel Type', inplace=True)
# gather them back
display = pd.concat([display0, display1, display2, display3], axis=1)

display

,Display Size,Resolution: width,Resolution: height,Refresh Rate,Panel Type
0,17.3,2560.0,1440.0,240.0,IPS + G-Sync
1,16.0,2560.0,1600.0,240.0,IPS + FreeSync
2,16.0,2560.0,1600.0,240.0,IPS + FreeSync
3,16.0,2560.0,1600.0,240.0,IPS + FreeSync
4,17.3,2560.0,1440.0,240.0,IPS + G-Sync
...,...,...,...,...,...
289166,17.3,1920.0,1080.0,60.0,IPS
289167,17.3,1600.0,900.0,60.0,TN
289168,17.3,1600.0,900.0,60.0,TN
289169,17.3,1600.0,900.0,60.0,TN


In [6]:
import re

# HDD/SDD: including capacity, type
disk = pd1['HDD/SSD'].str.split('+' ,expand=True)[[0, 1]]
part_1 = disk[0].astype(str)
part_2 = disk[1].astype(str)
result = np.zeros((len(part_1), 5))

def extract(lst):
    if lst.find('tb') != -1:
        res = re.findall(r'(\d+)\s?tb', lst)
        num = float(res[0]) * 1000
    elif lst.find('gb') != -1:
        res = re.findall(r'(\d+)\s?gb', lst)
        num = float(res[0])
    else:
        num = -1
    if lst.find('ssd') != -1:
        tp = 'SSD'
    elif lst.find('hdd') != -1:
        tp = 'HDD'
    elif lst.find('sshd') != -1:
        tp = 'SSHD'
    elif lst.find('optane') != -1:
        tp = 'Optane'
    else:
        tp = -1
    return num, tp

for i in range(len(part_1)):
    n = 0
    t = []
    lst = part_1.iloc[i].lower()
    if lst != 'None':
        a, b = extract(lst)
        if a != -1:
            n += a
        if b != -1:
            t.append(b)
    lst = part_2.iloc[i].lower()
    if lst != 'None':
        a, b = extract(lst)
        if a != -1:
            n += a
        if b != -1:
            t.append(b)
    if n != 0:
        result[i][0] = n
    # fill na for disk capacity
    else:
        result[i][0] = 512
    for j in t:
        match j:
            case 'SSD':
                result[i][1] = 1
            case 'HDD':
                result[i][2] = 1
            case 'SSHD':
                result[i][3] = 1
            case 'Optane':
                result[i][4] = 1
    if not t:
        # fill na for disk type
        result[i][1] = 1
        result[i][2] = 0
        result[i][3] = 0
        result[i][4] = 0
    
result = pd.DataFrame(result, columns=['Disk Capacity', 'SSD', 'HDD', 'SSHD', 'Optane'])
result

,Disk Capacity,SSD,HDD,SSHD,Optane
0,2000.0,1.0,0.0,0.0,0.0
1,16000.0,1.0,0.0,0.0,0.0
2,2000.0,1.0,0.0,0.0,0.0
3,2000.0,1.0,0.0,0.0,0.0
4,2000.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...
289166,256.0,1.0,0.0,0.0,0.0
289167,256.0,1.0,0.0,0.0,0.0
289168,2256.0,1.0,1.0,0.0,0.0
289169,1512.0,1.0,1.0,0.0,0.0


In [7]:
# RAM: including capacity, type
ram = pd1['RAM'].astype('str').str.split(',', expand=True)

capacity = ram[0].str.extract(r'(\d+)', expand=True).astype(float)
capacity.rename(columns={0: 'RAM Capacity'}, inplace=True)

ram_type = ram[1]
ram_type.rename('RAM Type', inplace=True)

ram = pd.concat([capacity, ram_type], axis=1)
ram

,RAM Capacity,RAM Type
0,64.0,DDR5
1,64.0,DDR5
2,32.0,DDR5
3,64.0,DDR5
4,16.0,DDR5
...,...,...
289166,8.0,DDR4
289167,16.0,DDR4
289168,16.0,DDR4
289169,8.0,DDR4


In [8]:
# OS
os = pd1['OS'].str.lower()
os

0          windows 11 pro
1          windows 10 pro
2          windows 11 pro
3          windows 10 pro
4         windows 11 home
               ...       
289166    windows 10 home
289167    windows 10 home
289168     windows 10 pro
289169    windows 10 home
289170    windows 10 home
Name: OS, Length: 289171, dtype: object

In [9]:
# Dimensions: We only concern about the depth
dim = pd1['Dimensions'].astype('str')
res = dim.str.extract(r'.*\s?x\s?.*\s?x\s?.*\s?x\s?(\d+\.?\d+)', expand=True).astype(float)
res = res.apply(lambda x: x*2.54)
dim = res.rename(columns={0: 'Dimension: Depth'})
dim

,Dimension: Depth
0,2.3368
1,2.0574
2,2.0574
3,2.0574
4,2.3368
...,...
289166,1.9812
289167,2.0574
289168,2.4892
289169,2.0574


In [10]:
# Body material
material = pd1['Body Material'].str.lower().astype('str')
res = {}

for row in material:
    row = row.lower()
    r = row.split(',')
    r = [x.strip() for x in r]
    for w in r:
        if w not in res:
            res[w] = len(res)

aaaaa = np.zeros((len(material), len(res)))
for i in range(len(material)):
    r = material.iloc[i]
    r = r.lower()
    r = r.split(',')
    r = [x.strip() for x in r]
    for w in r:
        aaaaa[i][res[w]] = 1
material = pd.DataFrame(aaaaa, columns=['Body material: ' + x for x in res.keys()])
material = material.drop('Body material: nan', axis=1)
material

,Body material: plastic / polycarbonate,Body material: aluminum,Body material: magnesium alloy,Body material: glass fiber,Body material: carbon,Body material: magnesium-lithium alloy,Body material: plastic,Body material: matte,Body material: black synthetic material
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
289166,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
289167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
289168,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
289169,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# weight: convert to float in kg
weight = pd1['Weight'].str.extract(r'(\d+\.?\d+)', expand=True).astype(float)
weight.rename(columns={0: 'Weight'}, inplace=True)
weight

,Weight
0,2.80
1,2.67
2,2.67
3,2.67
4,2.80
...,...
289166,2.60
289167,2.05
289168,2.45
289169,2.05


In [12]:
# NVME slot: Number of slots available
nvme = pd1['M.2 Slot'].astype('str')
res = nvme.str.extract(r'(\d+)\s?x', expand=True).astype(float)
nvme = res.rename(columns={0: 'Num of M.2 Slot'})
nvme.fillna(0, inplace=True)
nvme

,Num of M.2 Slot
0,2.0
1,2.0
2,2.0
3,2.0
4,2.0
...,...
289166,1.0
289167,0.0
289168,0.0
289169,0.0


In [13]:
# USB type C: number of USB type C ports
usb_c = pd1['USB Type-C'].astype('float')
usb_c.fillna(0, inplace=True)
usb_c

0         2.0
1         2.0
2         2.0
3         2.0
4         2.0
         ... 
289166    1.0
289167    1.0
289168    0.0
289169    1.0
289170    0.0
Name: USB Type-C, Length: 289171, dtype: float64

In [14]:
# USB type A: number of USB type A ports
usb_a = pd1['USB Type-A'].astype('float')
usb_a.fillna(0, inplace=True)
usb_a

0         2.0
1         2.0
2         2.0
3         2.0
4         2.0
         ... 
289166    3.0
289167    2.0
289168    3.0
289169    2.0
289170    3.0
Name: USB Type-A, Length: 289171, dtype: float64

In [15]:
# HDMI: generation of HDMI port
hdmi = pd1['HDMI'].astype('float')
hdmi.fillna(0, inplace=True)
hdmi

0         2.1
1         2.1
2         2.1
3         2.1
4         2.1
         ... 
289166    0.0
289167    1.4
289168    1.4
289169    1.4
289170    1.4
Name: HDMI, Length: 289171, dtype: float64

In [16]:
# Bluetooth: version of Bluetooth
bluetooth = pd1['Bluetooth'].astype('float')
bluetooth.fillna(0, inplace=True)
bluetooth

0         5.2
1         5.2
2         5.2
3         5.2
4         5.2
         ... 
289166    5.0
289167    5.2
289168    4.2
289169    5.2
289170    4.2
Name: Bluetooth, Length: 289171, dtype: float64

In [17]:
# Card reader: Have or have not
card_reader = pd1['Card Reader'].astype('str')
res = card_reader.apply(lambda x: 0 if x == 'nan' else (0 if x.lower() == '0' else 1))
card_reader = res.rename('Card Reader')
card_reader

0         0
1         1
2         1
3         1
4         0
         ..
289166    0
289167    0
289168    1
289169    0
289170    1
Name: Card Reader, Length: 289171, dtype: int64

In [18]:
# Ethernet/LAN: if there is a LAN port, then get the fastest speed, else 0
ethernet = pd1['Ethernet LAN'].astype('str')
for i in range(len(ethernet)):
    if ethernet.iloc[i] == 'None' or ethernet.iloc[i] == 'nan':
        ethernet.iloc[i] = np.nan
    else:
        res = re.findall(r'(\d+)', ethernet.iloc[i])
        ethernet.iloc[i] = res[-1]

ethernet = ethernet.astype(float)
ethernet

0         2500.0
1         2500.0
2         2500.0
3         2500.0
4         2500.0
           ...  
289166       NaN
289167       NaN
289168    1000.0
289169       NaN
289170    1000.0
Name: Ethernet LAN, Length: 289171, dtype: float64

In [19]:
# Security lock:
security = pd1['Security Lock slot'].astype('str')

for i in range(len(security)):
    if security.iloc[i] == 'None' or security.iloc[i] == 'nan' or security.iloc[i] == '0':
        security.iloc[i] = 0
    else:
        security.iloc[i] = 1

security = security.astype(float)
security

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
289166    0.0
289167    0.0
289168    1.0
289169    0.0
289170    1.0
Name: Security Lock slot, Length: 289171, dtype: float64

In [20]:
# Fingerprint reader
fingerprint = pd1['Fingerprint reader'].astype('float')
fingerprint.fillna(0, inplace=True)
fingerprint

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
289166    0.0
289167    0.0
289168    0.0
289169    0.0
289170    0.0
Name: Fingerprint reader, Length: 289171, dtype: float64

In [21]:
# Backlit keyboard
backlit = pd1['Backlit keyboard'].astype('str')

for i in range(len(backlit)):
    if backlit.iloc[i] == 'None' or backlit.iloc[i] == 'nan':
        backlit.iloc[i] = 0
    elif backlit.iloc[i] == '0':
        backlit.iloc[i] = 0
    else:
        backlit.iloc[i] = 1
        
backlit = backlit.astype(float)
backlit

0         1.0
1         1.0
2         1.0
3         1.0
4         1.0
         ... 
289166    0.0
289167    0.0
289168    0.0
289169    0.0
289170    0.0
Name: Backlit keyboard, Length: 289171, dtype: float64

In [22]:
# Cost
cost = pd1['Cost']
cost

0            2249
1         7389.99
2         3799.99
3         4584.99
4         2138.99
           ...   
289166        NaN
289167        NaN
289168        NaN
289169        NaN
289170        NaN
Name: Cost, Length: 289171, dtype: object

In [23]:
# score
score = pd1[['Total Score', 'Portability Score', 'Display Score', 'Work Score', 'Play Score']].astype(float)
score

,Total Score,Portability Score,Display Score,Work Score,Play Score
0,8.2,6.5,8.0,9.2,8.3
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,8.1,6.5,8.0,8.8,8.2
...,...,...,...,...,...
289166,NaN,NaN,NaN,NaN,NaN
289167,3.5,8.3,1.5,4.6,1.7
289168,3.4,7.3,1.5,4.6,1.7
289169,3.1,8.3,1.5,3.6,1.4


In [24]:
res = pd1[['link', 'name']]
cols = [res, display, result, ram, os, material, dim, weight, nvme, usb_c, usb_a, hdmi, bluetooth, card_reader, ethernet, security, fingerprint, backlit, score]

pd_out = pd.concat(cols, axis=1)
pd_out

,link,name,Display Size,Resolution: width,Resolution: height,Refresh Rate,Panel Type,Disk Capacity,SSD,HDD,...,Card Reader,Ethernet LAN,Security Lock slot,Fingerprint reader,Backlit keyboard,Total Score,Portability Score,Display Score,Work Score,Play Score
0,https://laptopmedia.com/laptop-specs/asus-rog-...,ASUS ROG Strix G17,17.3,2560.0,1440.0,240.0,IPS + G-Sync,2000.0,1.0,0.0,...,0,2500.0,0.0,0.0,1.0,8.2,6.5,8.0,9.2,8.3
1,https://laptopmedia.com/laptop-specs/asus-rog-...,ASUS ROG Zephyrus Duo 16,16.0,2560.0,1600.0,240.0,IPS + FreeSync,16000.0,1.0,0.0,...,1,2500.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN
2,https://laptopmedia.com/laptop-specs/asus-rog-...,ASUS ROG Zephyrus Duo 16,16.0,2560.0,1600.0,240.0,IPS + FreeSync,2000.0,1.0,0.0,...,1,2500.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN
3,https://laptopmedia.com/laptop-specs/asus-rog-...,ASUS ROG Zephyrus Duo 16,16.0,2560.0,1600.0,240.0,IPS + FreeSync,2000.0,1.0,0.0,...,1,2500.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN
4,https://laptopmedia.com/laptop-specs/asus-rog-...,ASUS ROG Strix G17,17.3,2560.0,1440.0,240.0,IPS + G-Sync,2000.0,1.0,0.0,...,0,2500.0,0.0,0.0,1.0,8.1,6.5,8.0,8.8,8.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289166,https://laptopmedia.com/laptop-specs/acer-aspi...,Acer Aspire 5,17.3,1920.0,1080.0,60.0,IPS,256.0,1.0,0.0,...,0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
289167,https://laptopmedia.com/laptop-specs/hp-17-5749/,HP 17,17.3,1600.0,900.0,60.0,TN,256.0,1.0,0.0,...,0,NaN,0.0,0.0,0.0,3.5,8.3,1.5,4.6,1.7
289168,https://laptopmedia.com/laptop-specs/hp-17-17-...,HP 17 (17-by4000),17.3,1600.0,900.0,60.0,TN,2256.0,1.0,1.0,...,1,1000.0,1.0,0.0,0.0,3.4,7.3,1.5,4.6,1.7
289169,https://laptopmedia.com/laptop-specs/hp-17-4594/,HP 17,17.3,1600.0,900.0,60.0,TN,1512.0,1.0,1.0,...,0,NaN,0.0,0.0,0.0,3.1,8.3,1.5,3.6,1.4


In [27]:
pd_out.to_csv('../data/ignore/laptop_cleaned.csv', index=False)